# Laboratory of automatic natural language processing 

## Introduction

This is notebook presenting results from NLP based study. 

THe main aim was creating texts classifier. Firstly, dataset for training was acquired
using webscrapper, then data was processed in order to reject dirty instances. After that,
the features of text was extracted. 

Each feature was word present in dataset, which was counted for each instance and then
the number of occurances of it. It was returned as vector of words.

After that, the classifier using Naive Bayes algorithm was trained and tested on
achieved data.

In [2]:
#Importing modules
from scraperFanFic import *
from sklearn.feature_extraction.text import (TfidfVectorizer,
                                             CountVectorizer)
from nltk.corpus import stopwords
import numpy as np
from nltk import FreqDist
import pandas as pd
from nltk.text import Text
from nltk.text import ContextIndex
from nltk.text import ConcordanceIndex

#Defining function for cleaning data
def preparing_stories(stories_list):
    result_list = list()
    for n in stories_list:
        #Getting rid of interpunction
        result = re.sub("[^a-zA-Z]"," ",n)
        #Lowering data to avoid problems wit case sensitivity
        result = result.lower()
        #Creating list of all words
        result = result.split(' ')
        #Rejecting words that occurs too often
        result = [w for w in result if w not in stopwords.words('english')]
        #Joining words into one string
        result = ' '.join(result)
        #Appending processed data to results
        result_list.append(result)
    return result_list

def aquire_freq_dist(stories_list):
    result_list = list()
    for n in stories_list:
        #Getting rid of interpunction
        result = re.sub("[^a-zA-Z]"," ",n)
        #Lowering data to avoid problems wit case sensitivity
        result = result.lower()
        #Creating list of all words
        result = result.split(' ')
        #Rejecting words that occurs too often
        result = [w for w in result if w not in stopwords.words('english')]
        #Joining words into one string
        for w in result:
            result_list.append(w)
    return FreqDist(result_list)


def aquire_list(stories_list):
    result_list = list()
    for n in stories_list:
        #Getting rid of interpunction
#         result = re.sub("[^a-zA-Z]"," ",n)
        #Lowering data to avoid problems wit case sensitivity
#         result = n.lower()
        #Creating list of all words
        result = n.split(' ')
        #Rejecting words that occurs too often
#         result = [w for w in result if w not in stopwords.words('english')]
        #Joining words into one string
        for w in result:
            result_list.append(w)
    return result_list

def array_words(stories_list):
    #Calling vectorizer
    count = CountVectorizer(analyzer='word',tokenizer=None,
                      preprocessor = None,max_features = 5000)
    stories = preparing_stories(stories_list)
    transformed = count.fit_transform(stories)
    feature_array = transformed.toarray()
    feature_name = count.get_feature_names()
    return feature_name,feature_array
from nltk.corpus import stopwords


In [3]:
#Getting datasets, 0 for Star Wars and 1 for Star Trek
#Code for main function is defined at the bottom of notebook,
#for aesthetic purposes.
# # dataset0,dataset1 = main()
X_train,X_test,y_test,y_train = main()

unprocessed = np.concatenate([X_train,X_test])
y = np.concatenate([y_train,y_test])
names,processed0 = array_words(unprocessed) 

# swdata = [X_train[n] for n in range(len(X_train)) if y_train[n] != 1]
# stdata = [X_train[n] for n in range(len(X_train)) if y_train[n] != 0]

# evenlist = np.concatenate([swdata[:200],stdata[:200]])

# sw_words = aquire_list(X_train)

# textsw = Text( sw_words)
# textst = Text(' '.join(stdata))

# even_words = aquire_list(evenlist)
# even_dist = aquire_freq_dist(evenlist)
# even_text = Text(even_words)

# first_words = np.array(even_dist.most_common()[1:][:10])[:,0]
# for n in first_words:
#     print(even_text.concordance(n,lines = 10))

## Training classifier

Now we are learning classifier and getting the results!

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

X_train,X_test,y_train,y_test = train_test_split(processed0,y)
gb = GaussianNB()
gb.fit(X_train,y_train)
gb.score(X_test,y_test)

0.9958847736625515

In [5]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
prec = precision_recall_fscore_support(y_test,gb.predict(X_test))
prec[0]
dict_res = {'Precision for class':np.round(prec[0],2),
            'Recall for class':np.round(prec[1],2),
            'F score':np.round(prec[2],2),
            'Number of instances':np.round(prec[3])}
results = pd.DataFrame(data=dict_res).T
results.columns = ['Star Wars','Star Trek']
results

,Star Wars,Star Trek
Precision for class,1.00,0.99
Recall for class,0.99,1.00
F score,0.99,1.00
Number of instances,95.00,148.00


In [1]:
def main():
    with open('testdata.txt','r') as test:
        X_test = test.read()
    
    with open('traindata.txt','r') as train:
        X_train = train.read()
    datatest = [n[9:] for n in X_train.split('\n')][:-1]
    datatrain = [n[9:] for n in X_test.split('\n')][:-1]
        
    dict_mapped = {'StarWars':0,'StarTrek':1}
    tartrain = [dict_mapped[X_train.split('\n')[n][:8]] for n in range(len(X_train.split('\n'))-1)]
    tartest = [dict_mapped[X_test.split('\n')[n][:8]] for n in range(len(X_test.split('\n'))-1)]
    return datatrain,datatest,tartrain,tartest

243